# Project 1, Part 6, Best Recommendation

University of California, Berkeley
Master of Information and Data Science (MIDS) program
w205 - Fundamentals of Data Engineering

Student: Landon Morin

Year: 2022

Semester: Spring

Section:


# Included Modules and Packages

Code cell containing your includes for modules and packages

In [21]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter
import matplotlib.ticker as mticker

import json

import os

import gmaps
import gmaps.geojson_geometries

from geographiclib.geodesic import Geodesic




import psycopg2

# Supporting code

Code cells containing any supporting code, such as connecting to the database, any functions, etc.  Remember you can use any code from the labs.

Number of zip codes: 550
['33004', '33009', '33010', '33012', '33013', '33014', '33015', '33016', '33018', '33019', '33020', '33021', '33023', '33024', '33025', '33026', '33027', '33028', '33029', '33032', '33033', '33039', '33054', '33055', '33056', '33109', '33122', '33125', '33126', '33127', '33128', '33129', '33130', '33131', '33132', '33133', '33134', '33135', '33136', '33137', '33138', '33139', '33140', '33141', '33142', '33143', '33144', '33145', '33146', '33147', '33149', '33150', '33154', '33155', '33156', '33157', '33158', '33160', '33161', '33162', '33165', '33166', '33167', '33168', '33169', '33170', '33172', '33173', '33174', '33175', '33176', '33177', '33178', '33179', '33180', '33181', '33182', '33183', '33184', '33185', '33186', '33187', '33189', '33190', '33193', '33196', '33301', '33312', '33314', '33315', '33316', '33317', '33324', '33325', '33328', '33330', '33331', '33332', '37013', '37014', '37015', '37027', '37035', '37048', '37062', '37064', '37066', '37067', '3

GeoJSON Files: ['ca_california_geojson.json', 'fl_florida_geojson.json', 'tn_tennessee_geojson.json', 'tx_texas_geojson.json', 'wa_washington_geojson.json']


FileNotFoundError: [Errno 2] No such file or directory: 'geojson_data/customer_zip_geojson.json'

In [11]:
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)

In [12]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [13]:
cursor = connection.cursor()

## The executives would like your best recommendation for the business. 

## Create an executive summary giving and explaining your best recommendation for the business. 

## You must support your summary with data, in the form of output of queries, data visualization, etc. There is a 1 query minimum.

In [14]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select c.customer_id,
       s.city as store,
       z.density, 
       z.longitude, 
       z.latitude, 
       z.zip
from stores as s
    join customers as c
        on s.store_id = c.closest_store_id
    join zip_codes as z
        on c.zip = z.zip
group by c.customer_id, z.zip, s.city, z.longitude, z.latitude, z.density
order by c.customer_id

"""

df1=my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df1

,customer_id,store,density,longitude,latitude,zip
0,1,Berkeley,12056.40,-122.2643,37.8343,94609
1,2,Berkeley,12056.40,-122.2643,37.8343,94609
2,3,Berkeley,12056.40,-122.2643,37.8343,94609
3,4,Berkeley,12056.40,-122.2643,37.8343,94609
4,5,Berkeley,12056.40,-122.2643,37.8343,94609
...,...,...,...,...,...,...
31077,31078,Nashville,220.15,-86.9277,35.8116,37179
31078,31079,Nashville,220.15,-86.9277,35.8116,37179
31079,31080,Nashville,220.15,-86.9277,35.8116,37179
31080,31081,Nashville,220.15,-86.9277,35.8116,37179


In [15]:
min_value = df1['density'].min()
max_value = df1['density'].max()

bins = np.linspace(min_value,max_value,10)

df1['bins'] = pd.cut(df1['density'], bins=bins, include_lowest=True)



In [16]:
df1

,customer_id,store,density,longitude,latitude,zip,bins
0,1,Berkeley,12056.40,-122.2643,37.8343,94609,"(11350.398, 17025.467]"
1,2,Berkeley,12056.40,-122.2643,37.8343,94609,"(11350.398, 17025.467]"
2,3,Berkeley,12056.40,-122.2643,37.8343,94609,"(11350.398, 17025.467]"
3,4,Berkeley,12056.40,-122.2643,37.8343,94609,"(11350.398, 17025.467]"
4,5,Berkeley,12056.40,-122.2643,37.8343,94609,"(11350.398, 17025.467]"
...,...,...,...,...,...,...,...
31077,31078,Nashville,220.15,-86.9277,35.8116,37179,"(0.259, 5675.329]"
31078,31079,Nashville,220.15,-86.9277,35.8116,37179,"(0.259, 5675.329]"
31079,31080,Nashville,220.15,-86.9277,35.8116,37179,"(0.259, 5675.329]"
31080,31081,Nashville,220.15,-86.9277,35.8116,37179,"(0.259, 5675.329]"


In [27]:
f = open('gmap_api_key.txt', 'r')
my_api_key = f.read()
f.close()

gmaps.configure(api_key=my_api_key)

In [50]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select z.latitude, z.longitude
from customers as cu
     join zip_codes as z
         on cu.zip = z.zip
order by 1,2

"""

df2 = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df2

,latitude,longitude
0,25.4837,-80.4136
1,25.5021,-80.3997
2,25.5021,-80.3997
3,25.5021,-80.3997
4,25.5302,-80.3919
...,...,...
31077,47.9581,-122.4047
31078,47.9581,-122.4047
31079,47.9581,-122.4047
31080,47.9581,-122.4047


In [52]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select latitude, longitude
from stores
order by 1

"""

df3 = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df3
df_markers = df3[['latitude','longitude']]

In [53]:
f = open('geojson_data/customer_zip_geojson.json')
customer_zip_geojson = json.load(f)
f.close()

kansas = (38.4942,-98.3132)

fig = gmaps.figure(center=kansas, zoom_level=4, map_type = 'HYBRID')
          
geojson_layer = gmaps.geojson_layer(customer_zip_geojson)

heatmap_layer = gmaps.heatmap_layer(df2)

marker_layer = gmaps.marker_layer(df_markers)

fig.add_layer(marker_layer)
fig.add_layer(heatmap_layer)
fig.add_layer(geojson_layer)

fig

Figure(layout=FigureLayout(height='420px'))